# TENSORFLOW 

TensorFlow es una plataforma de código abierto para crear y entrenar modelos de aprendizaje automático.

- **Sistema de computación simbólica**
- **Grafos computacionales**: estructura que representa operaciones matemáticas y el flujo de datos entre ellas. Esta formada por:
  - Nodos → representan operaciones (como suma, multiplicación).
  - Aristas → representan los datos que fluyen entre operaciones (tensores).
    x ----|
          | → + → z
    y ----|


### ¿Por qué no ejecutamos operaciones directamente (como en Python)?

1. **Podemos optimizar el grafo antes de ejecutarlo**: Antes de ejecutar las operaciones TensorFlow las analiza y las mejora automáticamente. Por ejemplo si tenemos `a = (3 * 1) + 0` TensorFlow se da cuenta de que es mejor escribir `a = 3`.
2. **Se puede ejecutar en varios dispositivos**: TensorFlow puede correr el código en diferentes hardware de procesamiento:
    - CPU (procesador normal) → lo que tiene cualquier ordenador.
    - GPU (tarjeta gráfica) → es mucho más rápida para operaciones grandes, como entrenar modelos de IA.
    - TPU (unidad de procesamiento de tensores) → un chip especial de Google aún más rápido para tareas específicas de Machine Learning.

    #### ACTIVIDAD: Investiga como funciona una TPU y contesta a las preguntas de Notion [Enlace para la investigación](https://cloud.google.com/tpu/docs/system-architecture-tpu-vm?hl=es-419)
3. **Es más fácil de guardar, exportar y reutilizar**: Cuando creas un grafo, además creas una representación completa del proceso. Cuando guardas un grafo (por ejemplo, con @tf.function o un modelo completo), TensorFlow guarda:
    - Las operaciones (suma, multiplicación, capas, etc.).
    - Los tensores que fluyen entre operaciones.
    - Los pesos del modelo si es un modelo entrenado.
    - El orden de ejecución y dependencias.
    - Las firmas de entrada/salida.

    Todo esto se guarda en formatos especiales, como:
    - **SavedModel/** (carpeta que contiene todo el grafo y sus pesos).
    - Archivos `.pb` (protobufs, para el grafo en sí).
    - Archivos `.ckpt` (para checkpoints de pesos).

| Guardar archivo `.py`                               | Guardar grafo/modelo en TensorFlow                               |
|------------------------------------------------------|-------------------------------------------------------------------|
| Guarda código fuente en texto plano.                | Guarda la estructura del grafo (operaciones, tensores) y pesos.  |
| Necesita Python y TensorFlow para volver a usarlo.  | Puede usarse en otros entornos (C++, móvil, servidor, etc).      |
| El código se tiene que volver a ejecutar.           | El grafo ya está listo para ejecutar.                            |
| No incluye pesos ni estado del modelo.              | Incluye todo: arquitectura + pesos entrenados.                   |


4. **Se puede visualizar (usando TensorBoard) para depurar o comprender el modelo**: TensorBoard es una herramienta visual incluida en TensorFlow que te permite ver lo que está pasando dentro de tu modelo:
    - Ver qué operaciones hay.
    - Ver qué datos fluyen entre ellas.
    - Detectar errores o cuellos de botella.

### Eager Execution

Desde TensorFlow 2, la ejecución inmediata (**Eager Execution**) está activada por defecto, lo que significa que cuando hacemos algo como:

```python

x = tf.constant(3.0)
y = tf.constant(4.0)
z = x + y
print(z)

```
Esto se ejecuta línea por línea, como lo haría en Python puro.

In [ ]:
import tensorflow as tf
print(f"tf version:", tf.__version__)  # Muestra la versión instalada

# Definir dos constantes (tensores)
a = tf.constant(5.0, name="constante_a")
b = tf.constant(3.0, name="constante_b")

# Realizar la suma (operación)
suma = tf.add(a, b)

# Mostrar el resultado
print("El resultado de la suma es:", suma.numpy())
# Nota: En TensorFlow 2.x, no es necesario crear una sesión explícitamente porque se utiliza la "eager execution" por defecto, las operaciones se ejecutan de inmediato y no es necesario usar 'sess.run()'.

print(f"Está activado EAGER EXECUTION:", tf.executing_eagerly()) # Verificar si la ejecución ansiosa está habilitada

tf version: 2.16.2
El resultado de la suma es: 8.0
Está activado EAGER EXECUTION: True



### Graph Execution

**Graph Execution** convierte la función en un grafo computacional interno, listo para ser optimizado y ejecutado más eficientemente.

- **Grafo Computacional**: Un grafo computacional es una red de operaciones conectadas entre sí. Cada nodo del grafo es una operación, y las flechas son los datos (tensores) que fluyen entre ellas.
- **`@tf.function`**: Cada vez que usamos `@tf.function`, TensorFlow convierte la función en un grafo: detecta las operaciones, conecta los nodos, y crea un “mapa” del cálculo.

| Tipo de ejecución                    | ¿Cómo funciona?                                | Ventaja                     | Ejemplo de uso                        |
|-------------------------------------|------------------------------------------------|-----------------------------|---------------------------------------|
| **Eager Execution**                 | Ejecuta operación por operación como en Python | Fácil de depurar y entender | `x + y` directamente                  |
| **Graph Execution** (`@tf.function`) | Compila funciones en grafos antes de ejecutar  | Rápido y eficiente          | Modelos de producción o entrenamiento |


In [ ]:
import tensorflow as tf

@tf.function # Decorador para compilar la función en un grafo de TensorFlow
def suma(a, b):
    return a + b # Operación que crea un nodo de tipo Add en el grafo

# Llamamos la función para que se genere el grafo
resultado = suma(tf.constant(2.0), tf.constant(3.0))

# Mostramos el resultado
print("Resultado:", resultado.numpy())


# TensorBoard

- TensorBoard es un dashboard web interactivo creado por Google para TensorFlow.
- Permite visualizar la evolución y comportamiento de modelos de Machine Learning durante el entrenamiento y la evaluación.
- Funciona leyendo archivos de registro (logs) que genera TensorFlow con datos del entrenamiento.


- versión A:
    - Usa una clase que hereda de tf.Module, organiza la lógica en un objeto.
    - Instancias de tf.Module pueden guardarse fácilmente con `tf.saved_model.save()`, exportando tanto grafo como pesos.
    - Más código, más formalismo, adecuado para proyectos o modelos.
    - Especifica la forma y tipo exactos de entrada para el grafo, asegurando compatibilidad y mejor optimización.


In [ ]:
import tensorflow as tf

# 1. Creamos un grafo de TensorFlow utilizando tf.Module
class Suma(tf.Module): # Clase que hereda de tf.Module para crear un modelo, permitiendo guardar y exportar funciones como un grafo
    @tf.function(input_signature=[ #input_signature define los tipos y formas de los tensores de entrada
        tf.TensorSpec(shape=(), dtype=tf.float32), # TensorSpec define la forma y tipo de los tensores de entrada
        tf.TensorSpec(shape=(), dtype=tf.float32)
    ])
    def __call__(self, a, b): # "Cuando alguien use este objeto como si fuera una función, se llamará a este método"
        return a + b

# 2. Crear una instancia del modelo
modelo = Suma()

# 3. Registrar el grafo para su visualización en TensorBoard
writer = tf.summary.create_file_writer("logs_suma")

# 4. Iniciar el seguimiento del grafo y registrar las operaciones
tf.summary.trace_on(graph=True)

# 5. Llamar al modelo con dos tensores para generar el grafo
modelo(tf.constant(2.0), tf.constant(3.0))

#6. Exportar el grafo a TensorBoard
with writer.as_default():
    tf.summary.trace_export(name="suma_grafo", step=0)

#Ejecutar el comando para ver el grafo: tensorboard --logdir=logs_suma
# Luego, abre tu navegador y ve a http://localhost:6006 para ver el grafo.


- versión B:
 - Define una función decorada con `@tf.function`.
 - La función se puede usar para crear grafos, pero no es un modelo completo para guardar.
 - Más adecuado para funciones sencillas, operaciones aisladas.
 - No tiene input_signature explícita, puede ser más flexible pero menos optimizable.

In [ ]:
import tensorflow as tf

# 1. Definir una función que realiza una suma y convertirla en un grafo de TensorFlow
@tf.function
def suma(a, b):
    return a + b

# 2. Registrar el grafo para su visualización en TensorBoard
writer = tf.summary.create_file_writer("logs")

# 3. Iniciar el seguimiento del grafo y registrar las operaciones
tf.summary.trace_on(graph=True, profiler=False) # Iniciar el seguimiento del grafo, registrando las operaciones y el grafo de ejecución, pero desactivando el perfilador para evitar la recopilación de estadísticas de rendimiento.

# 4. Llamar a la función con dos tensores para generar el grafo
suma(tf.constant(2.0, name="constante a"), tf.constant(3.0, name="constante b"))

# 5. Exportar el grafo a TensorBoard
with writer.as_default():
    tf.summary.trace_export(name="grafo_suma", step=0)

#Ejecutar el comando para ver el grafo: tensorboard --logdir=logs
# Luego, abre tu navegador y ve a http://localhost:6006 para ver el grafo.


2.16.2


### Guardar un Grafo

Puedes guardar el grafo e indicar la ruta donde guardar el grafo o modelo:

```python
tf.saved_model.save(modelo, "mi_modelo_guardado/")
```

- `tf` → Es el módulo principal de TensorFlow.
- `saved_model` → Es un submódulo de TensorFlow para trabajar con modelos guardados.
- `save(...)` → Es una función que guarda el modelo en el disco.
- `modelo` → Es el objeto del modelo que quieres guardar (puede ser un tf.keras.Model).
- `"mi_modelo_guardado"` → Es la ruta (carpeta) donde se guardará el modelo. Si no existe, se crea.

Esto crea una carpeta llamada mi_modelo_guardado/ con:

```css
mi_modelo_guardado/
├── saved_model.pb        ← El grafo computacional guardado
├── assets/               ← (vacío o con archivos extra, si se usaron)
└── variables/
    ├── variables.data-00000-of-00001  ← Pesos del modelo
    └── variables.index                ← Índices para restaurar los pesos
```

Para cargarlo posteriormente usariamos: 

```python
modelo_cargado = tf.saved_model.load("mi_modelo_guardado/")
```

¿Qué tipo de objetos podemos guardar?

- Modelos creados con tf.keras.Model.
- Funciones decoradas con @tf.function.
- Modelos personalizados construidos con TensorFlow.

In [17]:
import tensorflow as tf

# 1. Creamos un grafo de TensorFlow utilizando tf.Module
class Suma(tf.Module): # Clase que hereda de tf.Module para crear un modelo, permitiendo guardar y exportar funciones como un grafo
    @tf.function(input_signature=[ #input_signature define los tipos y formas de los tensores de entrada
        tf.TensorSpec(shape=(), dtype=tf.float32), # TensorSpec define la forma y tipo de los tensores de entrada
        tf.TensorSpec(shape=(), dtype=tf.float32)
    ])
    def __call__(self, a, b): # "Cuando alguien use este objeto como si fuera una función, se llamará a este método"
        return a + b

# 2. Crear una instancia del modelo
modelo = Suma()

# 3. Registrar el grafo para su visualización en TensorBoard
writer = tf.summary.create_file_writer("logs_suma")

# 4. Iniciar el seguimiento del grafo y registrar las operaciones
tf.summary.trace_on(graph=True)

# 5. Llamar al modelo con dos tensores para generar el grafo
modelo(tf.constant(2.0), tf.constant(3.0))

#6. Exportar el grafo a TensorBoard
with writer.as_default():
    tf.summary.trace_export(name="suma_grafo", step=0)

# 7. GUARDAR EL MODELO CON SU GRAFO
tf.saved_model.save(modelo, "modelo_guardado_suma")


INFO:tensorflow:Assets written to: modelo_guardado_suma/assets


INFO:tensorflow:Assets written to: modelo_guardado_suma/assets


### Versión 1 de Tensorflow

In [ ]:
"""import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
# 1. Definir el grafo computacional
# # Crear nodos constantes
a = tf.constant(5.0, name="constante_a")
b = tf.constant(3.0, name="constante_b")
# Crear nodo de suma
suma = tf.add(a, b, name="operacion_suma")
# 2. Crear una sesión
with tf.Session() as sess:   
     # 3. Ejecutar el grafo    
     resultado = sess.run(suma)    
     # 4. Imprimir el resultado    
     print("El resultado de la suma es:", resultado)
     # La sesión se cierra automáticamente al salir del bloque 'with'"""

Explicación del Código:
- **import tensorflow as tf**: Importa la biblioteca TensorFlow y la asigna el alias tf.
- **a = tf.constant(5.0, name="constante_a")**: Crea un nodo constante en el grafo que contiene el valor 5.0. El name es opcional, pero ayuda a identificar los nodos en TensorBoard.
- **b = tf.constant(3.0, name="constante_b")**: Crea un nodo constante en el grafo que contiene el valor 3.0.
- **suma = tf.add(a, b, name="operacion_suma")**: Crea un nodo en el grafo que representa la operación de suma entre los nodos a y b.
- **with tf.Session() as sess:**: Crea una sesión de TensorFlow. El bloque with asegura que la sesión se cierre correctamente al finalizar.
- **resultado = sess.run(suma)**: Ejecuta el grafo, comenzando en el nodo suma. El resultado de la operación se almacena en la variable resultado.
- **print("El resultado de la suma es:", resultado)**: Imprime el resultado de la suma en la consola.